In [1]:
import pandas as pd
import numpy as np

In [2]:
def clean_columns(df):
    df.columns = df.columns.str.strip()
    return df

In [3]:
ddos_data_path = "../data/Hybrid_IDS2017_2018_2019/final_dataset.csv"
ftp_brute_force_data_path = "../data/CICIDS_2018/BruteForce_FTP.csv"
ssh_brute_force_data_path = "../data/CICIDS_2018/BruteForce_SSH.csv"

In [8]:
SAMPLE_SIZE = 500000
CHUNK_SIZE = 100000
ddos_chunks = []
benign_chunks = []

print("DDoS Verileri Okunuyor...")
for chunk in pd.read_csv(ddos_data_path, chunksize = CHUNK_SIZE, low_memory = False):
    chunk = clean_columns(chunk)

    filtered_ddos = chunk[chunk['Label'] == 'ddos']
    filtered_benign = chunk[chunk['Label'] == 'Benign']

    if ((not (filtered_ddos.empty)) and (len(filtered_ddos)) <= SAMPLE_SIZE):
        ddos_chunks.append(filtered_ddos)

    if ((not (filtered_benign.empty)) and (len(filtered_benign)) <= SAMPLE_SIZE):
        benign_chunks.append(filtered_benign)

    if (len(ddos_chunks) > SAMPLE_SIZE * 3) and (len(benign_chunks) > SAMPLE_SIZE * 3):
        break
print("DDoS verileri okundu!")
print(f"DDoS verileri toplam -> {len(ddos_chunks)} satır.")
print(f"Benign verileri toplam -> {len(benign_chunks)} satır.")

DDoS Verileri Okunuyor...
DDoS verileri okundu!
DDoS verileri toplam -> 65 satır.
Benign verileri toplam -> 64 satır.


In [7]:
print("Brute Force Verileri Yükleniyor...")
df_brute_force_ftp = pd.read_csv(ftp_brute_force_data_path)
df_brute_force_ssh = pd.read_csv(ssh_brute_force_data_path)
print("Brute Force Verileri Yüklendi!\nİşleme Aşamasına Geçiliyor...")

df_brute_force_ftp = clean_columns(df_brute_force_ftp)
df_brute_force_ssh = clean_columns(df_brute_force_ssh)

df_brute_force_raw = pd.concat([df_brute_force_ftp, df_brute_force_ssh])

bf_attacks = df_brute_force_raw[df_brute_force_raw['Label'].isin(['FTP-BruteForce', 'SSH-Bruteforce'])].copy()

print("İşleme Aşaması Tamamlandı!")

len_bf_attacks = len(bf_attacks)
print(f"Brute Force Saldırıları Toplam Satır Sayısı: {len_bf_attacks}")

Brute Force Verileri Yükleniyor...
Brute Force Saldırıları Toplam Satır Sayısı: 380949


In [13]:
print("DDoS Verileri İşleniyor...")
if ddos_chunks:
    ddos_attacks = pd.concat(ddos_chunks, axis = 0)
else:
    print("Hata: Hiç DDoS Verisi Bulunamadı!")
print("DDoS Verileri İşlendi!")
len_ddos_attacks = len(ddos_attacks)
print(f"DDoS Saldırıları Toplam Satır Sayısı: {len_ddos_attacks}")

print("Benign Verileri İşleniyor...")
if benign_chunks:
    ddos_benign = pd.concat(benign_chunks, axis = 0)
else:
    print("Hata: Hiç Benign Verisi Bulunamadı!")
print("Benign Verileri İşlendi!")

len_ddos_benign = len(ddos_benign)
print(f"Bening Verileri Toplam Satır Sayısı: {len_ddos_benign}")

NEW_SAMPLE_SIZE = min(len_ddos_attacks, len_bf_attacks, len_ddos_benign)
print(f"Yeni Örneklem Sayımız: {NEW_SAMPLE_SIZE}")

DDoS Verileri İşleniyor...
DDoS Verileri İşlendi!
DDoS Saldırıları Toplam Satır Sayısı: 6472647
Benign Verileri İşleniyor...
Benign Verileri İşlendi!
Bening Verileri Toplam Satır Sayısı: 6321980
Yeni Örneklem Sayımız: 380949


In [14]:
print("Veri Setlerinden Örneklemler Seçiliyor...")
if len(ddos_benign) >= NEW_SAMPLE_SIZE:
    ddos_benign = ddos_benign.sample(n = NEW_SAMPLE_SIZE, random_state = 0)
    print(f"Benign Verileri Hazır -> Toplam {len(ddos_benign)} satır.")
else:
    print(f"İstenen Sayıya Ulaşılamadı. Toplam Benign Verisi: {len_ddos_benign}")

if len(ddos_attacks) >= NEW_SAMPLE_SIZE:
    ddos_attacks = ddos_attacks.sample(n = NEW_SAMPLE_SIZE, random_state = 0)
    print(f"DDoS Verileri Hazır -> Toplam {len(ddos_attacks)} satır.")
else:
    print(f"İstenen Sayıya Ulaşılamadı. Toplam DDoS Verisi: {len(ddos_attacks)}")

if len(bf_attacks) >= NEW_SAMPLE_SIZE:
    bf_attacks = bf_attacks.sample(n = NEW_SAMPLE_SIZE, random_state = 0)
    print(f"Brute Force Verileri Hazır -> Toplam {len(bf_attacks)} satır.")
else:
    print(f"İstenen Sayıya Ulaşılamadı. Toplam Brute Force Verisi: {len(bf_attacks)}")
print("Örneklem Seçme İşlemi Başarılı Bir Şekilde Tamamlandı!")

Veri Setlerinden Örneklemler Seçiliyor...
Benign Verileri Hazır -> Toplam 380949 satır.
DDoS Verileri Hazır -> Toplam 380949 satır.
Brute Force Verileri Hazır -> Toplam 380949 satır.
Örneklem Seçme İşlemi Başarılı Bir Şekilde Tamamlandı!


In [15]:
ddos_benign.head(2)

,Unnamed: 0,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
7929445,1810547,172.31.67.82-186.95.218.106-445-54987-6,172.31.67.82,445,186.95.218.106,54987,6,20/02/2018 10:52:27,138,2,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
11795366,2649290,172.31.64.87-96.231.60.219-3389-55187-6,96.231.60.219,55187,172.31.64.87,3389,6,20/02/2018 04:32:57,3100677,8,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [16]:
ddos_attacks.head(2)

,Unnamed: 0,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
5061506,2210841,172.31.69.25-18.219.193.20-80-38906-6,172.31.69.25,80,18.219.193.20,38906,6,16/02/2018 11:18:00 PM,4553032,4,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos
3432013,581348,172.31.69.25-18.219.193.20-80-56948-6,18.219.193.20,56948,172.31.69.25,80,6,16/02/2018 11:18:50 PM,58090,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos


In [17]:
bf_attacks.head(2)

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
404444,21,6,2,1,1,0,0.0,0,0,0.0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FTP-BruteForce
407792,21,6,1,1,1,0,0.0,0,0,0.0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FTP-BruteForce


In [25]:
print(f"DDoS Verisi İçerisindeki Sınıflar:\n{ddos_attacks['Label'].value_counts()}\n")
print(f"Benign Verisi İçerisindeki Sınıflar:\n{ddos_benign['Label'].value_counts()}\n")
print(f"Brute Force Verisi İçerisindeki Sınıflar:\n{bf_attacks['Label'].value_counts()}")

DDoS Verisi İçerisindeki Sınıflar:
Label
ddos    380949
Name: count, dtype: int64

Benign Verisi İçerisindeki Sınıflar:
Label
Benign    380949
Name: count, dtype: int64

Brute Force Verisi İçerisindeki Sınıflar:
Label
FTP-BruteForce    193360
SSH-Bruteforce    187589
Name: count, dtype: int64


In [26]:
bf_attacks['Label'] = 'BruteForce'

In [27]:
print(f"DDoS Verisi İçerisindeki Sınıflar:\n{ddos_attacks['Label'].value_counts()}\n")
print(f"Benign Verisi İçerisindeki Sınıflar:\n{ddos_benign['Label'].value_counts()}\n")
print(f"Brute Force Verisi İçerisindeki Sınıflar:\n{bf_attacks['Label'].value_counts()}")

DDoS Verisi İçerisindeki Sınıflar:
Label
ddos    380949
Name: count, dtype: int64

Benign Verisi İçerisindeki Sınıflar:
Label
Benign    380949
Name: count, dtype: int64

Brute Force Verisi İçerisindeki Sınıflar:
Label
BruteForce    380949
Name: count, dtype: int64


In [28]:
ddos_benign['Label'] = 0 # Normal Trafik 0
ddos_attacks['Label'] = 1 # DDoS Saldırısı 1
bf_attacks['Label'] = 2 # Brute Force Saldırısı 2

In [29]:
target_features = [
    'Destination Port', 'Flow Duration', 
    'Total Fwd Packets', 'Total Backward Packets', 
    'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 
    'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std',
    'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std',
    'Flow Bytes/s', 'Flow Packets/s', 
    'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
    'Fwd IAT Total', 'Bwd IAT Total', 
    'Fwd Header Length', 'Bwd Header Length', 
    'Fwd Packets/s', 'Bwd Packets/s', 
    'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
    # Flags
    'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 
    'ACK Flag Count', 'URG Flag Count', 'ECE Flag Count', 
    'Down/Up Ratio', 'Average Packet Size', 
    'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 
    'Label'
]

column_mapping = {
    'Dst Port': 'Destination Port',
    'Tot Fwd Pkts': 'Total Fwd Packets',
    'Tot Bwd Pkts': 'Total Backward Packets',
    'TotLen Fwd Pkts': 'Total Length of Fwd Packets',
    'TotLen Bwd Pkts': 'Total Length of Bwd Packets',
    'Fwd Pkt Len Max': 'Fwd Packet Length Max',
    'Fwd Pkt Len Min': 'Fwd Packet Length Min',
    'Fwd Pkt Len Mean': 'Fwd Packet Length Mean',
    'Fwd Pkt Len Std': 'Fwd Packet Length Std',
    'Bwd Pkt Len Max': 'Bwd Packet Length Max',
    'Bwd Pkt Len Min': 'Bwd Packet Length Min',
    'Bwd Pkt Len Mean': 'Bwd Packet Length Mean',
    'Bwd Pkt Len Std': 'Bwd Packet Length Std',
    'Flow Byts/s': 'Flow Bytes/s',
    'Flow Pkts/s': 'Flow Packets/s',
    'Fwd IAT Tot': 'Fwd IAT Total',
    'Bwd IAT Tot': 'Bwd IAT Total',
    'Fwd Header Len': 'Fwd Header Length',
    'Bwd Header Len': 'Bwd Header Length',
    'Fwd Pkts/s': 'Fwd Packets/s',
    'Bwd Pkts/s': 'Bwd Packets/s',
    # Paket Boyutları
    'Pkt Len Min': 'Min Packet Length',
    'Pkt Len Max': 'Max Packet Length',
    'Pkt Len Mean': 'Packet Length Mean',
    'Pkt Len Std': 'Packet Length Std',
    'Pkt Len Var': 'Packet Length Variance',
    'Pkt Size Avg': 'Average Packet Size',
    'FIN Flag Cnt': 'FIN Flag Count',
    'SYN Flag Cnt': 'SYN Flag Count',
    'RST Flag Cnt': 'RST Flag Count',
    'PSH Flag Cnt': 'PSH Flag Count',
    'ACK Flag Cnt': 'ACK Flag Count',
    'URG Flag Cnt': 'URG Flag Count',
    'ECE Flag Cnt': 'ECE Flag Count',
    'CWE Flag Count': 'CWE Flag Count', # eklemekten zarar gelmez
    'Init Fwd Win Byts': 'Init_Win_bytes_forward',
    'Init Bwd Win Byts': 'Init_Win_bytes_backward'
}

In [30]:
def standardize_and_select(df, features, mapping):
    df.columns = df.columns.str.strip()
    
    # Sadece sözlükteki isimleri değiştir
    df.rename(columns = mapping, inplace = True)
    
    # Sadece hedef listemizde olan sütunları seç
    # Eğer dataframe'de hedef sütun yoksa, hata vermemesi için kontrol etmeliyiz
    available_features = [c for c in features if c in df.columns]
    
    # Eksik sütunları raporla
    missing = set(features) - set(available_features)
    if missing:
        print(f"UYARI: Şu sütunlar bu dosyada bulunamadı ve atlanacak: {missing}")
    
    return df[available_features].copy()

In [31]:
print(f"Standartlaştırmadan önceki benign sütun sayısı: {len(ddos_benign.columns)}")
ddos_benign = standardize_and_select(ddos_benign, target_features, column_mapping)
print(f"Standartlaştırmadan sonraki benign sütun sayısı: {len(ddos_benign.columns)}")

print(f"\nStandartlaştırmadan önceki ddos sütun sayısı: {len(ddos_attacks.columns)}")
ddos_attacks = standardize_and_select(ddos_attacks, target_features, column_mapping)
print(f"Standartlaştırmadan sonraki ddos sütun sayısı: {len(ddos_attacks.columns)}")

print(f"\nStandartlaştırmadan önceki brute force sütun sayısı: {len(bf_attacks.columns)}")
bf_attacks = standardize_and_select(bf_attacks, target_features, column_mapping)
print(f"Standartlaştırmadan sonraki brute force sütun sayısı: {len(bf_attacks.columns)}")

Standartlaştırmadan önceki benign sütun sayısı: 85
Standartlaştırmadan sonraki benign sütun sayısı: 43

Standartlaştırmadan önceki ddos sütun sayısı: 85
Standartlaştırmadan sonraki ddos sütun sayısı: 43

Standartlaştırmadan önceki brute force sütun sayısı: 79
Standartlaştırmadan sonraki brute force sütun sayısı: 43


In [33]:
def remove_duplicate_columns(df):
    return df.loc[:, ~df.columns.duplicated()]

print("Veriler Birleştiriliyor...")

print("\nVerilerdeki Tekrarlayan Sütunlar Kaldırılıyor...")
ddos_benign = remove_duplicate_columns(ddos_benign)
ddos_attacks = remove_duplicate_columns(ddos_attacks)
bf_attacks = remove_duplicate_columns(bf_attacks)
print("Verilerdeki Tekrarlayan Sütunlar Kaldırıldı!")

print("\nYeni DataFrame Objelerindeki Sütun Sayıları:\n")
print(f"Benign DataFrame Objesi Sütun Sayısı: {len(ddos_benign.columns)}")
print(f"\nDDoS DataFrame Objesi Sütun Sayısı: {len(ddos_attacks.columns)}")
print(f"\nBrute Force DataFrame Objesi Sütun Sayısı: {len(bf_attacks.columns)}")

print("\nİlgili DataFrame Objeleri Birleştiriliyor...")
final_cols = list(set(ddos_attacks.columns) & set(ddos_benign.columns) & set(bf_attacks.columns))

# Label sütununu en sona alalım
if 'Label' in final_cols:
    final_cols.remove('Label')
final_cols.append('Label')

final_df = pd.concat([
    ddos_benign[final_cols],
    ddos_attacks[final_cols],
    bf_attacks[final_cols]
], axis = 0)
print("DataFrame Objeleri Birleştirildi!")
print(f"\nYeni DataFrame Satır Sayısı: {len(final_df)}")
print(f"\nYeni DataFrame Sütun Sayısı: {len(final_df.columns)}")

Veriler Birleştiriliyor...

Verilerdeki Tekrarlayan Sütunlar Kaldırılıyor...
Verilerdeki Tekrarlayan Sütunlar Kaldırıldı!

Yeni DataFrame Objelerindeki Sütun Sayıları:

Benign DataFrame Objesi Sütun Sayısı: 43

DDoS DataFrame Objesi Sütun Sayısı: 43

Brute Force DataFrame Objesi Sütun Sayısı: 43

İlgili DataFrame Objeleri Birleştiriliyor...
DataFrame Objeleri Birleştirildi!

Yeni DataFrame Satır Sayısı: 1142847

Yeni DataFrame Sütun Sayısı: 43


In [34]:
final_df.head(2)

,Total Length of Bwd Packets,Bwd Packet Length Min,Max Packet Length,Fwd Packet Length Std,Average Packet Size,Total Length of Fwd Packets,RST Flag Count,Down/Up Ratio,Bwd IAT Total,SYN Flag Count,...,ECE Flag Count,Flow IAT Std,Fwd Packet Length Mean,Total Fwd Packets,URG Flag Count,Fwd Packets/s,Init_Win_bytes_backward,Fwd IAT Total,Fwd Header Length,Label
7929445,0.0,0.0,0.0,0.000000,0.000,0.0,0,0.0,0.0,0,...,0,0.000000,0.0,2,0,14492.753623,-1,138.0,40,0
11795366,1618.0,0.0,1173.0,228.129662,172.875,1148.0,1,1.0,2950036.0,0,...,1,280149.233266,143.5,8,0,2.580082,62852,3100677.0,172,0


In [35]:
save_path = "../data/final_hybrid_data/benign_ddos_bf_dataset.csv"
print(f"Final Veri Seti Şu Yola Kayıt Ediliyor -> {save_path}")
final_df.to_csv(save_path, index = False)
print(f"Final Veri Seti Başarılı Bir Şekilde Kayıt Edildi!\n{save_path}")

Final Veri Seti Şu Yola Kayıt Ediliyor -> ../data/final_hybrid_data/benign_ddos_bf_dataset.csv
Final Veri Seti Başarılı Bir Şekilde Kayıt Edildi!
../data/final_hybrid_data/benign_ddos_bf_dataset.csv


In [37]:
print(f"Final Veri Seti Sınıf Dağılımı\n")
print(final_df['Label'].value_counts())

Final Veri Seti Sınıf Dağılımı

Label
0    380949
1    380949
2    380949
Name: count, dtype: int64
